In [ ]:
from ccdproc import ImageFileCollection

In [ ]:
from pathlib import Path

In [ ]:
object_name = "OB0006"

In [ ]:
DATADIR = Path(f"../imaging/{object_name}")

In [ ]:
def get_paths(dirname, data_dir=DATADIR):
    """Given the image superdirectory, returns the subdirectories corresponding to the 
    bias, flat, standard, and object images respectively
    """
    bias_dir = data_dir.joinpath(dirname, "bias")
    flat_dir = data_dir.joinpath(dirname, "flat")
    std_dir = data_dir.joinpath(dirname, "stds")
    obj_dir = data_dir.joinpath(dirname, "object")

    return bias_dir, flat_dir, std_dir, obj_dir

In [ ]:
allimgs_order = ["bias", "flat", "std", "obj"]

In [ ]:
def get_image_files(dirname, data_dir):
    """Given an image superdirectory, retrieve ImageFileCollections
    for each of the subdirectories of images: bias, flat, stds, and object
    """
    paths = get_paths(dirname, data_dir)
    return [ImageFileCollection(path) for path in paths]

In [ ]:
for ob in range(1, 10):
    print(f"OB000{ob}")
    print("-"*100)
    print()
    allimgs = get_image_files("raw", Path(f"OB000{ob}"))
    for (imgs, name) in zip(allimgs, allimgs_order):
        print(name)
        imgs.summary[["file", "filter2", "ra", "dec", "exptime", "opentime"]].pprint(max_width=120)
        print()

    print("\n\n")